- to do multiple transactions in psql:

```
begin;
insert into test(num, data) values(123, 'HELLO PSQL');
insert into test(num, data) values(123, 'HELLO PSQL');
commit;
```

## Connection Pooling

- connection pools are cached database connections
- those connections are created and maintained to get reused
- main benefit: performance improvement

Connection pooling is the process of having a pool of active connections on the backend servers. 
- These can be used any time a user sends a request.
- The server will assign an active connection to the user.

# Usage in Python
- Python has support for working with different databases:
1. sqlite
2. mysql
3. postgres
4. etc.

## Psycopg
- Psycopg is the most popular PostgreSQL database adapter for python
- Psycopg 2 is mostly implemented in C
- Many Python types are supported and adapted to matching PostgreSQL data types

### Basic module usage

> python3 -m pip install psycopg2

> create database python_db;

In [1]:
import psycopg2

#Connect to an existing database
conn = psycopg2.connect(
    host="localhost", #or
    #host="127.0.0.1", 
    database="python_db",
    user='postgres',
    password='password'
)

#Open a cursor to perform database operations
cursor = conn.cursor()

#Execute a command: this creates a new table

# try:
#     cursor.execute("create table test (id serial primary key, num integer, data varchar(50))")
# except Exception as e:
#     print('Error:', e)

cursor.execute("create table if not exists test (id serial primary key, num integer, data varchar(50))")

#Pass data to fill a query placeholders 
cursor.execute("insert into test (num, data) values (%s, %s)", (100, "abc'def"))

#Query the database and obtain data as python objects 
cursor.execute("select * from test;")
data = cursor.fetchall()

# Make the change to the database persistent
conn.commit()

#Close communication wih the database
cursor.close()
#conn.close()
data



[(1, 100, "abc'def")]

- connect() creates a new database session

In [2]:
print(type(conn))

<class 'psycopg2.extensions.connection'>


- each connection/session can create new cursor instances
- cursor instances can execute database commands
- commands are sent by the following methods:
1. execute()
2. executemany()
- to retrieve data use the following cursor methods:
1. fetchall() - gets all row of the query
2. fetchone() - gets one row of the query
3. fetchmany() - gets as many rows as specified e.g. fetchmany(5)

### Access Data From a cursor
- You can have many cursors sharing the same connection to a database.
- Cursors created from the same connection are not isolated
- i.e. any changes done to the database by a cursor are visible by other cursor

- a query can potentially match very large sets of data
- Read operations do not immediately return all values matching the query
- these operations rely on cursors

- cursors fetches data in batches to reduce memory consumption

- Closing the cursor frees resources associated to the queries.
- It would not eliminate the connection to the database itself.
(Therefore, there is no need for reauthentication)



### Connections and Cursors
- Connections and cursors can be used as context managers and commits if no exception occurs. 
- the connection is not closed by the context
- the cursor is closed by the context

In [3]:
with conn:
    with conn.cursor() as curs:
        curs.execute("CREATE TABLE if not exists test3 (id serial PRIMARY KEY, num integer, data varchar)")
        curs.execute('select * from test')
        data = curs.fetchall()
        print(curs)
print(curs)
#conn.commit happens
#curs.close() happens
data


<cursor object at 0x7effb674b130; closed: 0>
<cursor object at 0x7effb674b130; closed: -1>


[(1, 100, "abc'def")]

### Passing parameters to SQL queries
### execute and executemany()
- Execute a database operation 
- Parameters may be provided 
- For positional variables binding, the second argument must always be a sequence

In [4]:
with conn:
    with conn.cursor() as curs:
        curs.execute("""
        insert into test (num, data)
        values (%s, %s)
        """, (22, 'BLA'))
        curs.execute("select * from test")
        print(curs.fetchall())

[(1, 100, "abc'def"), (2, 22, 'BLA')]


- Named arguments are supported too using %(name)s placeholders in the query
- allows to specify the values in any order and to repeat the same value

In [5]:
with conn:
    with conn.cursor() as curs:
        curs.execute("""
        insert into test (num, data)
        values (%(value1)s, %(value2)s)
        """, {'value1':100, 'value2':'DATA'})
        curs.execute("select * from test")
        print(curs.fetchall())

[(1, 100, "abc'def"), (2, 22, 'BLA'), (3, 100, 'DATA')]


### The problem with the query parameters - SQL Injection

In [6]:
with conn:
   with conn.cursor() as curs:
      SQL = "insert into test (data) values ('%s');" # don't use quotes around %s
      #user_input = "Bla'); select * from test where '1' in ('1"
      user_input = input('please give data to store')
      curs.execute(SQL % user_input) #NEVER DO THIS
      print(curs.fetchall())


ProgrammingError: no results to fetch

In [ ]:
'Hello %s' % 'bla'

'Hello bla'

In [ ]:
with conn:
   with conn.cursor() as curs:
      SQL = "insert into test (data) values (%s);" # don't use quotes around %s
      user_input = "HELLO WORLD'); select * from test where '1' in ('1"
      #user_input = 'HELLO WORLD'
      #user_input = input('please give data to store')
      curs.execute(SQL, (user_input,)) # DO IT THIS WAY
      

<span style=color:red;>Warning: Never, never, NEVER use Python string concatenation (+) or string parameters interpolation (%) to pass variables to a SQL query string. </span>

In [ ]:
SQL = "insert into test (data) values (%s);"

![](exploits_of_a_mom.png)

### executemany



In [ ]:
with conn:
    with conn.cursor() as curs:
        number = (1000,)
        numbers = [(1000,), (2000,), (3000,)]
        SQL = "insert into test (num) values (%s)"
        # curs.execute(SQL, number) #single insert
        curs.executemany(SQL, numbers) # multiple inserts 



- Parameters are bounded to the query using the same rules described in the execute() method.
- In its current implementation this method is not faster than executing execute() in a loop

## Adaptation of PYthon values to sql types
There is default mapping to convert Python types into PostgreSQL types, and vice versa

- None <-->NULL
- bool <-->bool
- decimal <--> numeric
- str <--> varchar
- float <--> real or double
- int <--> smallint or integer or bigint
- date <--> date
- time <--> timetz


### mogrify
- Return a query string after arguments binding. 
- The string returned is exactly the one that would be sent by execute() method.


In [12]:
with conn:
    with conn.cursor() as curs:
        query1 = curs.mogrify("insert into test (num, data) values (10, 'bal  ')") 
        query2 = curs.mogrify("insert into test (num, data) values (%s, %s)", (10, 'BLA')) 
        #query2 = curs.execute("insert into test (num, data) values (%s, %s)", (10, 'BLA')) 
        print(query2)

b"insert into test (num, data) values (10, 'BLA')"


- the return value is always a byte string

### Constants adaptation
- Python None, True and False are converted into the proper SQL literals:


In [13]:
with conn:
    with conn.cursor() as curs:
        query = curs.mogrify("""
        select %s, %s, %s;
        """, (None, True, False))
        print(query)

b'\n        select NULL, true, false;\n        '


In [18]:
### Numeric adaptation
from decimal import Decimal

print(Decimal("10.00"))
print(float("10.00"))

with conn:
    with conn.cursor() as curs:
        query = curs.mogrify("""
        select %s, %s, %s;""", (10, 10.00, Decimal("10.00")))
        print(query)

10.00
10.0
b'\n        select 10, 10.0, 10.00;'


- sometimes you may prefer to receive numeric data as float instead:
- you can configure an adapter to cast PostgreSQL numeric to Python float.

In [23]:
### Date/Time objects adaptation

from datetime import datetime

dt = datetime.now()
dt
with conn:
    with conn.cursor() as curs:
        query = curs.mogrify("""
        select %s, %s, %s;
        """, (dt, dt.date(), dt.time()))
        print(query)

b'select column from test '


### List adaptation

In [24]:
with conn:
    with conn.cursor() as curs: 
        query = curs.mogrify("select %s;", ([10,20,30], ))
        print(query)

b'select ARRAY[10,20,30];'


### Transactions

- Transactions are handled by the connection class
- the first time a command is sent to the database, a new transaction is created.
- All db commands will be executed in the context of the same transaction
- all cursors created by the same connection share the same transaction

Should any command fail, the transaction will be aborted
- no further command will be executed until rollback is called.

In [25]:
conn1 = psycopg2.connect(
    host="localhost",
    database="python_db",
    user="postgres",
    password="password"
)

conn2 = psycopg2.connect(
    host="localhost",
    database="python_db",
    user="postgres",
    password="password"
)

In [52]:
curs1 = conn1.cursor()
curs2 = conn1.cursor()
curs2_1 = conn2.cursor()
#conn1.autocommit = False

#conn1.rollback()
try:
    curs1.execute('select * from bla')
except Exception as e:
    conn1.commit()
    print(e)


curs2.execute('select * from test') #FROM CONNECTION 1
print(curs2.fetchall())

# curs2_1.execute('select * from test') #FROM CONNECTION 2
# print(curs2_1.fetchall())
conn1.commit()

relation "bla" does not exist
LINE 1: select * from bla
                      ^

[(1, 100, "abc'def"), (2, 22, 'BLA'), (3, 100, 'DATA'), (5, 10, 'BLA')]


- Postgres relies heavily on transactions to keep data consistent across concurrent connections and parallel activities.
- transactions allow a database to implement the ACID properties:
the db must be able:
1. to handle units of work on its whole (Atomicity)
2. to store data without inter-mixed changes to the data (consistency)
3. to store data in a way that concurrency actions are executed as if they were
alone (isolation) 
4. to store data in a permanent way (Durability)

- An atomic transaction is an indivisible and irreducible series of database operations
- either all operatons occur, or nothing occurs

- every action issued against the database is executed within a transaction
- any transaction is assigned a unique number: *xid*
- you can get the xid by inspecting the hidden column *xmin*


In [62]:
SQL_xid = 'select txid_current()'
SQL_insert = "insert into test(num, data) values(123, 'HALLO TRANSACTION')"
SQL_xmin = 'select xmin, txid_current(), * from test'

In [58]:
curs1.execute(SQL_xid)
print(curs1.fetchall())
conn1.commit()

[(3079,)]


In [71]:
conn1.rollback()
conn2.rollback()
#From CONNECTION 2
curs2_1.execute("insert into test (data) values('CONNECTION2')")
#From CONNECTION 1
curs1.execute("insert into test (data) values('CONNECTION1')")
curs2.execute("insert into test (data) values('CONNECTION1')")
conn1.commit()
conn2.commit()
curs2_1.execute(SQL_xmin)
curs2_1.fetchall()


[('3070', 3101, 1, 100, "abc'def"),
 ('3072', 3101, 2, 22, 'BLA'),
 ('3073', 3101, 3, 100, 'DATA'),
 ('3075', 3101, 5, 10, 'BLA'),
 ('3080', 3101, 6, 123, 'HALLO'),
 ('3080', 3101, 7, 123, 'HALLO'),
 ('3082', 3101, 10, 123, 'HALLO'),
 ('3082', 3101, 11, 123, 'HALLO'),
 ('3083', 3101, 13, 123, 'HALLO'),
 ('3083', 3101, 14, 123, 'HALLO'),
 ('3084', 3101, 16, 123, 'HALLO TRANSACTION'),
 ('3084', 3101, 17, 123, 'HALLO TRANSACTION'),
 ('3085', 3101, 19, 123, 'HALLO TRANSACTION'),
 ('3085', 3101, 20, 123, 'HALLO TRANSACTION'),
 ('3088', 3101, 22, None, 'CONNECTION1'),
 ('3088', 3101, 23, None, 'CONNECTION1'),
 ('3091', 3101, 25, None, 'CONNECTION1'),
 ('3091', 3101, 26, None, 'CONNECTION1'),
 ('3096', 3101, 30, None, 'CONNECTION2'),
 ('3097', 3101, 31, None, 'CONNECTION1'),
 ('3097', 3101, 32, None, 'CONNECTION1'),
 ('3099', 3101, 33, None, 'CONNECTION2'),
 ('3100', 3101, 34, None, 'CONNECTION1'),
 ('3100', 3101, 35, None, 'CONNECTION1')]

## Connection Pooling

- connection pools are cached database connections
- those connections are created and maintained to get reused
- main benefit: performance improvement

Connection pooling is the process of having a pool of active connections on the backend servers. 
- These can be used any time a user sends a request.
- The server will assign an active connection to the user.

![](pooling.png)

### Example SimpleConnectionPool

```
conn = psycopg2.connect(
    host="localhost",
    database="python_db",
    user="postgres",
    password="password"
)
```

In [16]:
from psycopg2.pool import SimpleConnectionPool

pool = SimpleConnectionPool(minconn=1, maxconn=2, 
                            host="localhost",
                            database="python_db",
                            user="postgres",
                            password="password")

In [9]:
SQL_active = "select datname from pg_stat_activity"
SQL_killall = """
select pg_terminate_backend(pid) from pg_stat_activity where datname='python_db'
 """

In [19]:
conn = pool.getconn()
curs = conn.cursor()
curs.execute(SQL_active)
curs.fetchall()

PoolError: connection pool exhausted